Load all the necessary classes

In [ ]:
from eocanvas import API, Credentials
from eocanvas.api import Input, Config, ConfigOption
from eocanvas.processes import SnapProcess
from eocanvas.snap.graph import Graph

First, save your credentials. They will be automatically loaded when required.

In [ ]:
c = Credentials(username="germano", password="12qwaszx!")
c.save()

The easiest way to interact with the service is to use high level objects, such as SnapProcess or DataTailorProcess.

Let's see a SnapProcess example.
Set the inputs for SNAP.
The graph can be either loaded from a file (suggested) or build programmatically.
The URL for the input can be retrieved by using the HDA Python client.

In [ ]:
from hda import Client

c = Client()

q = {
  "dataset_id": "EO:EUM:DAT:SENTINEL-3:OL_2_WFR___",
  "dtstart": "2024-07-05T09:28:00.000Z",
  "dtend": "2024-07-05T09:30:00.000Z",
  "timeliness": "NT"
}

r = c.search(q)
url = r.get_download_urls()[0]

In [ ]:
graph = Graph.from_uri("olci_binding.xml")
config = Config(key="img1", options=ConfigOption(uncompress=True, sub_path="xfdumanifest.xml"))
inputs = Input(key="img1", url=url)
inputs = Input(key="img1", url="http://gateway.impl.wekeo2.eu/hda-broker/api/v1/dataaccess/download/66b37374b6a632e1f39b3058")

Create a SnapProcess object passing all the required arguments.

In [ ]:
process = SnapProcess(snap_graph=graph, eo_config=config, eo_input=inputs)

If you want, you can explore how the inputs are being formatted before the submission to the API.

In [ ]:
process.prepare_inputs()

Calling Process.run, the process will be submitted to the API and it will block until the results have been downloaded.

In [ ]:
process.run()

Let's show another example using EUMETSAT Data Tailor.
The code is very similar.

First, we import the necessary classes:

In [ ]:
from eocanvas.datatailor.chain import Chain
from eocanvas.processes import DataTailorProcess

We can load a DataTailor chain from a YAML file.
The inputs are configured exactly as in the SNAP case:

In [ ]:
chain = Chain.from_file("olci_resample.yaml")
inputs = Input(key="img1", url="http://gateway.impl.wekeo2.eu/hda-broker/api/v1/dataaccess/download/66c357dcb6a632e1f39b3131")

process = DataTailorProcess(epct_chain=chain, epct_input=inputs)

In [ ]:
process.run()

Alternatively, it is possible to directly use the API as shown below: 

In [ ]:
api = API()

In [ ]:
# shows the available processes and their configuration
processes = api.get_processes()
print(processes)

In [ ]:
# as seen above
process = SnapProcess(snap_graph=graph, eo_config=config, eo_input=inputs)

# the exec_process method returns a job instance that can be monitored
job = api.exec_process(process)
print(job.status)

# reload the status from the API
job.refresh_from_api()

In [ ]:
# the application log (either SNAP or DataTailor) can be retrieved as well
api.get_job_logs(job=job)

In [ ]:
# once the job status is "successful", the results are available to download
results = api.get_job_results(job=job)
results[0].download()